In [1]:
import shutil
import os

# Clean up corrupted TFDS cache if it exists
food101_dir = os.path.expanduser("~/.tensorflow_datasets/food101")
if os.path.exists(food101_dir):
    shutil.rmtree(food101_dir)
    print("🧹 Removed corrupted food101 cache.")
else:
    print("✅ No previous food101 cache found.")

✅ No previous food101 cache found.


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, applications
from tensorflow.keras.utils import image_dataset_from_directory

IMG_SIZE = 224
BATCH = 32


In [3]:
def preprocess(img, label, training=True):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    if training:
        img = tf.image.random_flip_left_right(img)
    return img, label


In [4]:
import logging
tf.get_logger().setLevel(logging.ERROR)

train_ds, val_ds = tfds.load(
    "food101",
    split=["train[:90%]", "train[90%:]"],
    as_supervised=True
)

train_ds = (train_ds
    .shuffle(10000)
    .map(lambda x, y: preprocess(x, y, training=True), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH)
    .prefetch(tf.data.AUTOTUNE))

val_ds = (val_ds
    .map(lambda x, y: preprocess(x, y, training=False), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH)
    .prefetch(tf.data.AUTOTUNE))



/Users/connorcorrigan/Desktop/plastic2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                  | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|█████████████████████████| 1/1 [00:00<00:00, 110.45 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|                              | 0/4996278331 [00:00<?, ? MiB/s]

Dl Size...: 100%|███| 4996278331/4996278331 [00:00<00:00, 335950338081.16 MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Generating splits...:   0%|                          | 0/2 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 3308 examples [00:01, 3307.34 examples/s]
Generating train examples...: 6824 examples [00:02, 3430.03 examples/s]
Generating train examples...: 10255 examples [00:03, 3392.69 examples/s]
Generating train examples...: 13900 examples [00:04, 3491.51 examples/s]
Generating train examples...: 17663 exampl

Dataset food101 downloaded and prepared to /Users/connorcorrigan/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [13]:
# Step 1: Load dataset from directory
fruits_ds = image_dataset_from_directory(
    "fruits360/Training",
    image_size=(IMG_SIZE, IMG_SIZE),
    label_mode="int",
    batch_size=BATCH,
    shuffle=True
)

# ✅ Step 2: Extract class names *before* mapping
fruit_class_names = fruits_ds.class_names

# Step 3: Shift and cast labels to match Food-101
NUM_FOOD101_CLASSES = 101
fruits_ds = fruits_ds.map(lambda x, y: (
    tf.cast(x, tf.float32) / 255.0,
    tf.cast(y + NUM_FOOD101_CLASSES, tf.int64)
))


Found 102790 files belonging to 201 classes.


In [14]:
full_train_ds = train_ds.concatenate(fruits_ds)


In [15]:
TOTAL_CLASSES = NUM_FOOD101_CLASSES + len(fruit_class_names)

base_model = applications.EfficientNetB0(
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    weights="imagenet",
    pooling="avg"
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Dropout(0.2),
    layers.Dense(TOTAL_CLASSES, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 302)            │       386,862 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,436,433 (16.92 MB)

 Trainable params: 386,862 (1.48 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [16]:
EPOCHS = 5
history = model.fit(full_train_ds, validation_data=val_ds, epochs=EPOCHS)


Epoch 1/5


2025-06-01 22:40:45.987127: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2356s 439ms/step - accuracy: 0.0089 - loss: 4.9189 - val_accuracy: 0.0000e+00 - val_loss: 11.6335
Epoch 2/5
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2230s 416ms/step - accuracy: 0.0091 - loss: 5.0739 - val_accuracy: 0.0000e+00 - val_loss: 11.6321
Epoch 3/5
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2195s 410ms/step - accuracy: 0.0094 - loss: 5.0727 - val_accuracy: 0.0000e+00 - val_loss: 11.6290
Epoch 4/5
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2122s 396ms/step - accuracy: 0.0099 - loss: 5.0717 - val_accuracy: 0.0000e+00 - val_loss: 11.6208
Epoch 5/5
5344/5344 ━━━━━━━━━━━━━━━━━━━━ 2098s 392ms/step - accuracy: 0.0088 - loss: 5.0769 - val_accuracy: 0.0000e+00 - val_loss: 11.6101


In [18]:
model.export("food_model")

converter = tf.lite.TFLiteConverter.from_saved_model("food_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open("food101_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at 'food_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_239')
Output Type:
  TensorSpec(shape=(None, 302), dtype=tf.float32, name=None)
Captures:
  5748151824: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  5748147024: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  5730730256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730736784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730736016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730737552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730736592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730737360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5729089296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730739664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5730738512: TensorSpec(shape=(), dtype=tf.reso

W0000 00:00:1749246246.847395 42048674 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1749246246.848174 42048674 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-06 17:44:06.851627: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: food_model
2025-06-06 17:44:06.880661: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-06 17:44:06.880679: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: food_model
I0000 00:00:1749246246.960985 42048674 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-06-06 17:44:06.974476: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-06 17:44:07.445601: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: food_model
2025-06-06 17:44:07.578718: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags {